### CybORG Action Space

Without the use of wrappers,  CybORG actions need to be constructed by the agent before being passed in. If you are not interested this we suggest you skip to the wrapper tutorial.

The action space is updated every step and can be found as a dictionary in the results object. Because this dictionary is quite large, we will only print the keys below.

In [74]:
import random

import inspect
from os.path import dirname
from pprint import pprint

from CybORG import CybORG
from CybORG.Simulator.Scenarios import FileReaderScenarioGenerator

path = inspect.getfile(CybORG)
path = dirname(path) + f'/Simulator/Scenarios/scenario_files/Scenario1b.yaml'
sg = FileReaderScenarioGenerator(path)
env = CybORG(scenario_generator=sg)

results = env.reset(agent='Red')
action_space = results.action_space
pprint(action_space.keys())

dict_keys(['action', 'subnet', 'ip_address', 'session', 'username', 'password', 'process', 'port', 'target_session', 'agent', 'hostname'])


The CybORG action space is divided into "actions" and "parameters". Actions represent the use of specific cyber tools (for example a network scanning tool like nmap), while parameters represent the inputs the tool requires to function (to scan the interfaces of a host with nmap, you need to provide the ip address of the host).

The "actions" are located under the 'action' key in the action_space dictionary.

In [75]:
actions = action_space['action']
pprint(actions)

{<class 'CybORG.Simulator.Actions.Action.Sleep'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverRemoteSystems.DiscoverRemoteSystems'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.DiscoverNetworkServices.DiscoverNetworkServices'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.ExploitRemoteService.ExploitRemoteService'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.PrivilegeEscalate.PrivilegeEscalate'>: True,
 <class 'CybORG.Simulator.Actions.AbstractActions.Impact.Impact'>: True}


We can see that our actions are each custom classes that form the keys of the above dictionary. The values specify whether this action is currently valid. In Scenario 1b, this value will always be True.

The remaining keys in the scenario dictionary represent different classes of parameters. For example, if we examine the 'ip_address' key we will get a dictionary whose keys are the various ip_addresses on the network. The values are again booleans, which represents whether Red knows about this ip_address or not.

In [76]:
ips = action_space['ip_address']
pprint(ips)

{IPv4Address('10.0.127.65'): False,
 IPv4Address('10.0.127.71'): False,
 IPv4Address('10.0.127.73'): False,
 IPv4Address('10.0.127.76'): False,
 IPv4Address('10.0.127.78'): False,
 IPv4Address('10.0.192.180'): False,
 IPv4Address('10.0.192.181'): False,
 IPv4Address('10.0.192.184'): False,
 IPv4Address('10.0.192.185'): False,
 IPv4Address('10.0.192.187'): False,
 IPv4Address('10.0.232.17'): False,
 IPv4Address('10.0.232.20'): False,
 IPv4Address('10.0.232.21'): True,
 IPv4Address('10.0.232.25'): False,
 IPv4Address('10.0.232.26'): False,
 IPv4Address('10.0.232.29'): False}


To construct an action, we choose (or import) an action class, then instantiate it by passing in the necessary parameters.

In [77]:
import random
from CybORG.Simulator.Actions import DiscoverNetworkServices
unknown_ips = [ip for ip in ips if not ips[ip]]
ip = random.choice(unknown_ips)

action = DiscoverNetworkServices(session=0,agent='Red',ip_address=ip)

We have deliberately chosen to scan an ip address that Red Agent doesn't know about. Although randomly guessing an ip address to scan is possible in the real world, we have decided it is out of scope for our current implementation and so this action will always fail. If you want to expose your agent to the action space, you should filter out all parameters with False values first.

In [78]:
results = env.step(action=action,agent='Red')
print(results.observation)

{'success': <TrinaryEnum.UNKNOWN: 2>}


### Red Actions

We will now take a detailed look at Red Team's actions and understand what they do. Red's actions are listed below.

In [79]:
pprint([action.__name__ for action in actions if actions[action]])

['Sleep',
 'DiscoverRemoteSystems',
 'DiscoverNetworkServices',
 'ExploitRemoteService',
 'PrivilegeEscalate',
 'Impact']


The Sleep action does nothing and requires no parameters.

In [80]:
from CybORG.Simulator.Actions import *

action = Sleep()
results = env.step(action=action,agent='Red')
print(results.observation)

{'success': <TrinaryEnum.UNKNOWN: 2>}


The DiscoverRemoteSystems action represents a ping sweep and takes in a subnet parameter to return all ips active on that subnet. Note how we pull the 

In [81]:
subnets = action_space['subnet']
known_subnets = [subnet for subnet in subnets if subnets[subnet]]
subnet = known_subnets[0]

action = DiscoverRemoteSystems(subnet = subnet, session=0,agent='Red')
results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.232.17': {'Interface': [{'IP Address': IPv4Address('10.0.232.17'),
                                'Subnet': IPv4Network('10.0.232.16/28')}]},
 '10.0.232.20': {'Interface': [{'IP Address': IPv4Address('10.0.232.20'),
                                'Subnet': IPv4Network('10.0.232.16/28')}]},
 '10.0.232.21': {'Interface': [{'IP Address': IPv4Address('10.0.232.21'),
                                'Subnet': IPv4Network('10.0.232.16/28')}]},
 '10.0.232.25': {'Interface': [{'IP Address': IPv4Address('10.0.232.25'),
                                'Subnet': IPv4Network('10.0.232.16/28')}]},
 '10.0.232.26': {'Interface': [{'IP Address': IPv4Address('10.0.232.26'),
                                'Subnet': IPv4Network('10.0.232.16/28')}]},
 'success': <TrinaryEnum.TRUE: 1>}


The DiscoverNetworkServices action represents a port scan and takes in an ip address parameter to return a list of open ports and their respective services. These will be represented in the observation as new connections. The Red team must have discovered the ip address using the DiscoverRemoteSystems action in order for this action to succeed.

In [82]:
known_ips = [ip for ip in ips if ips[ip]]
ip = random.choice(known_ips)
action = DiscoverNetworkServices(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.232.21': {'Interface': [{'IP Address': IPv4Address('10.0.232.21')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.232.21'),
                                                 'local_port': 22}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.232.21'),
                                                 'local_port': 21}]}]},
 'success': <TrinaryEnum.TRUE: 1>}


The ExploitRemoteService represents the use of a service exploit to obtain a reverse shell on the host. It requires an ip address as an input parameter and creates a new shell on the target host. 

CybORG actually models several different types of real-world exploits and this action chooses between them depending on the services available and the operating system of the host. This action will only ever succeed if the host's ip address has been discovered by Red team.

Usually the shell created by this action will be a shell with user privileges, but some exploits, such as EternalBlue, give SYSTEM access to a windows machine. In this case, performing the Privilege Escalation action afterwards is unnecessary, although our rules-based agents always will.

In [83]:
action = ExploitRemoteService(ip_address=ip,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'10.0.232.21': {'Interface': [{'IP Address': IPv4Address('10.0.232.21')}],
                 'Processes': [{'Connections': [{'Status': <ProcessState.OPEN: 2>,
                                                 'local_address': IPv4Address('10.0.232.21'),
                                                 'local_port': 22}],
                                'Process Type': <ProcessType.SSH: 2>},
                               {'Connections': [{'local_address': IPv4Address('10.0.232.21'),
                                                 'local_port': 54265,
                                                 'remote_address': IPv4Address('10.0.232.21'),
                                                 'remote_port': 22}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.232.21'),
                                                 'local_port': 22,
                                                 'remote_address': IPv4Address('10.0.232.21'),
                     

The PrivilegeEscalate represents the use of malware to establish a privileged shell with root (Linux) or SYSTEM (Windows) privileges. This action requires a user shell to be on the target host.

This action has the potential to reveals information about hosts on other subnets, which can then be scanned and exploited.

In [84]:
hostname = results.observation[str(ip)]['System info']['Hostname']
action = PrivilegeEscalate(hostname=hostname,session=0,agent='Red')

results = env.step(action=action,agent='Red')
pprint(results.observation)

{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.232.21'),
                          'Interface Name': 'eth0',
                          'Subnet': IPv4Network('10.0.232.16/28')}],
           'Processes': [{'PID': 3387, 'Username': 'SYSTEM'}],
           'Sessions': [{'Agent': 'Red',
                         'ID': 0,
                         'PID': np.int64(3387),
                         'Timeout': 0,
                         'Type': <SessionType.RED_ABSTRACT_SESSION: 10>,
                         'Username': 'SYSTEM'}]},
 'success': <TrinaryEnum.TRUE: 1>}


The Impact action represents the degredation of services. It requires a hostname input parameter, but will only work on the 'OpServer0' host on the Operational subnet and needs to be continually run in order to have an ongoing effect.

In [85]:
from CybORG.Agents import B_lineAgent

results = env.reset(agent='Red')
obs = results.observation
action_space = results.action_space
agent = B_lineAgent()

while True:
    action = agent.get_action(obs,action_space)
    results = env.step(action=action,agent='Red')
    obs = results.observation
    
    if action.__class__.__name__ == 'Impact':
        print(action)
        print(obs)
        break

Impact Op_Server0
{'success': <TrinaryEnum.TRUE: 1>}


## Blue Actions

We will now take a look at Blue Team's actions and how they interact with those of Red Team.

In [86]:
env = CybORG(sg, agents={'Red':B_lineAgent})
results = env.reset('Blue')
actions = results.action_space['action']

pprint([action.__name__ for action in actions if actions[action]])

['Sleep', 'Monitor', 'Analyse', 'Remove', 'Misinform', 'Restore']


Similar to Red Team, the sleep action for Blue Team has no effect. However, like all Blue Team actions it does have passive monitoring capabilities as explained in the observation tutorial.

In [87]:
action = Sleep()

for i in range(4):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 2:
        # The particular obs we want
        pprint(obs)

{'User0': {'Interface': [{'IP Address': IPv4Address('10.0.170.209')}],
           'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.170.209'),
                                           'remote_address': IPv4Address('10.0.170.214'),
                                           'remote_port': 443}]}],
           'System info': {'Architecture': <Architecture.x64: 2>,
                           'Hostname': 'User0',
                           'OSDistribution': <OperatingSystemDistribution.WINDOWS_SVR_2008: 4>,
                           'OSType': <OperatingSystemType.WINDOWS: 2>,
                           'OSVersion': <OperatingSystemVersion.W6_1_7601: 13>,
                           'position': (28, 30)}},
 'User3': {'Interface': [{'IP Address': IPv4Address('10.0.170.209')}],
           'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.170.209'),
                                           'remote_address': IPv4Address('10.0.170.214'),
                     

As explained by the Observation Tutorial, the Analyse action can detect malware files on a single host. This mimics the use of a malware-detection tool such as DensityScout. Like all of Blue's actions, it requires a hostname parameter.

We can see below that the action discovers malware on 'User1' as well as the passive monitoring picking up an exploit used Enterprise 1.

In [88]:
action = Analyse(hostname='User1',session=0,agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    if i == 1:
        pprint(obs)

{'Enterprise0': {'Interface': [{'IP Address': IPv4Address('10.0.214.42')}],
                 'Processes': [{'Connections': [{'local_address': IPv4Address('10.0.214.42'),
                                                 'local_port': 22,
                                                 'remote_address': IPv4Address('10.0.170.214'),
                                                 'remote_port': 59417}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.214.42'),
                                                 'local_port': 22,
                                                 'remote_address': IPv4Address('10.0.170.214'),
                                                 'remote_port': 59551}]},
                               {'Connections': [{'local_address': IPv4Address('10.0.214.42'),
                                                 'local_port': 22,
                                                 'remote_address': IPv4Address('10.0.170.214'),
     

The Remove action allows Blue Team to remove any of Red's user-level shells, simulating the act of killing it as a process. It will not remove a privileged shell. This is because privileged shells in Scenario1b are assumed to be persistent, meaning that if you remove them they will immediately come back.

We can see below that the Red agent attempts to PrivilegeEscalate, but this fails as its shell has been killed. The next turn it has to re-exploit the machine. Notice the use of the get_last_action method to work out what Red's last move was.

In [89]:
action = Remove(hostname='Enterprise1', session=0, agent='Blue')

for i in range(2):
    results = env.step(action=action,agent='Blue')
    obs = results.observation
    pprint(obs)
    print(73*'-')
    print(env.get_last_action('Red'))
    print(73*'*')

{'success': <TrinaryEnum.TRUE: 1>}
-------------------------------------------------------------------------
PrivilegeEscalate Enterprise0
*************************************************************************
{'success': <TrinaryEnum.TRUE: 1>}
-------------------------------------------------------------------------
DiscoverRemoteSystems 10.0.214.32/28
*************************************************************************


The Restore action represents reverting the system to a known baseline. This will restore a host to the state it was at the beginning of the game. This will wipe all of Red's shells away, with the notable exception of Red's starting host 'User0', which has been baselined into the system. Although Restore is more powerful than Remove, it necessarily causes some disruption on the network so has a large negative penalty associated by using it.

Below we can see that the Analyse action detects malware on 'User1', but this disappears after restore has been used.

In [90]:
for i in range(10):
    env.step() # So Red's actions don't interfere

action = Analyse(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)
    
action = Restore(hostname='User1', session=0, agent='Blue')
results = env.step(action=action,agent='Blue')
obs = results.observation
pprint(obs)

action = Analyse(hostname='User1', session=0, agent='Blue')
obs = results.observation
pprint(obs)

{'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.TRUE: 1>}
{'success': <TrinaryEnum.TRUE: 1>}


### Miscellany

If you create an action that doesn't make any sense within the current scenario, CybORG will accept it, but automatically convert it to an Invalid Action. These actions automatically give a reward of -0.1.

In [91]:
action = Analyse(hostname = "Uncle Ted's Macbook", session = 1.1, agent='Cyan')

results = env.step(action=action,agent='Blue')

print(results.action)
print(results.reward)

InvalidAction
-13.1
